<a href="https://colab.research.google.com/github/JustAyush/BookRecSys/blob/master/withClean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install lightfm



     |████████████████████████████████| 307kB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [0]:
# mounting google drive for datasets
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# importing necessary libraries

import pandas as pd
import numpy as np
from sklearn import preprocessing
from lightfm import LightFM
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix 
from scipy.sparse import coo_matrix 
import time
from datetime import datetime, timedelta
from sklearn.metrics import roc_auc_score
from lightfm.evaluation import auc_score
import pickle
import re
import json

In [0]:
# specifying the dataset paths

userPath = '/content/drive/My Drive/data/users.csv'
bookPath = '/content/drive/My Drive/data/books.csv'
ratingPath = '/content/drive/My Drive/data/ratings.csv'
# bookPath = '/content/drive/My Drive/data/cleanTwo.csv'

In [0]:
# Loading the dataset with pandas

users = pd.read_csv(userPath, sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

ratings = pd.read_csv(ratingPath, sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

books = pd.read_csv(bookPath, sep=';', error_bad_lines=False, encoding="latin-1", low_memory = False)
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\nSkipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\nSkipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\nSkipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [0]:
# publisher names ‘DK Publishing Inc’ and ‘Gallimard’ have been incorrectly loaded as yearOfPublication in dataset
# Following piece of code makes necessary correction for these rows and set the data type for yearOfPublication as int

books.loc[books.ISBN == '0789466953','yearOfPublication'] = 2000
books.loc[books.ISBN == '0789466953','bookAuthor'] = "James Buckley"
books.loc[books.ISBN == '0789466953','publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '0789466953','bookTitle'] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)"


books.loc[books.ISBN == '078946697X','yearOfPublication'] = 2000
books.loc[books.ISBN == '078946697X','bookAuthor'] = "Michael Teitelbaum"
books.loc[books.ISBN == '078946697X','publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '078946697X','bookTitle'] = "DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)"


books.loc[(books.ISBN == '0789466953') | (books.ISBN == '078946697X'),:]

books.loc[books.ISBN == '2070426769','yearOfPublication'] = 2003
books.loc[books.ISBN == '2070426769','bookAuthor'] = "Jean-Marie Gustave Le ClÃ?Â©zio"
books.loc[books.ISBN == '2070426769','publisher'] = "Gallimard"
books.loc[books.ISBN == '2070426769','bookTitle'] = "Peuple du ciel, suivi de 'Les Bergers"

books.yearOfPublication=pd.to_numeric(books.yearOfPublication, errors='coerce')

# removing the entries with publication year=0 and greater than 2006

books.loc[(books.yearOfPublication > 2006) | (books.yearOfPublication == 0),'yearOfPublication'] = np.NAN


books.yearOfPublication.fillna(round(books.yearOfPublication.mean()), inplace=True)

books.yearOfPublication.isnull().sum()


# Checking the entries with publisher=null

books.loc[books.publisher.isnull(),:]

# Filling those NaN values with 'other'

books.loc[(books.ISBN == '193169656X'),'publisher'] = 'other'
books.loc[(books.ISBN == '1931696993'),'publisher'] = 'other'

# Replacing the entries with age below 5 and above 90 with the mean value of Age and setting its type to int

users.loc[(users.Age > 90) | (users.Age < 5), 'Age'] = np.nan
users.Age = users.Age.fillna(users.Age.mean())
users.Age = users.Age.astype(np.int32)
users.Age.unique()

array([34, 18, 17, 61, 26, 14, 25, 19, 46, 55, 32, 24, 20, 23, 51, 31, 21,
       44, 30, 57, 43, 37, 41, 54, 42, 50, 39, 53, 47, 36, 28, 35, 13, 58,
       49, 38, 45, 62, 63, 27, 33, 29, 66, 40, 15, 60, 79, 22, 16, 65, 59,
       48, 72, 56, 67, 80, 52, 69, 71, 73, 78,  9, 64, 12, 74, 75, 76, 83,
       68, 11, 77, 70,  8,  7, 81, 10,  5,  6, 84, 82, 90, 85, 86, 87, 89,
       88])

In [0]:
# ratings dataset should have books only which exist in our books dataset

ratings_new = ratings[ratings.ISBN.isin(books.ISBN)]

# rows having book ISBN not part of books dataset are dropped off
print (ratings.shape)
print (ratings_new.shape)

users.shape

(1149780, 3)
(1031136, 3)


(278858, 3)

In [0]:
# Separating implicit and explicit datasets

ratings_explicit = ratings_new[ratings_new.bookRating != 0]
ratings_implicit = ratings_new[ratings_new.bookRating == 0]


In [0]:
# Assigning threshold for the number of users that have rated books and also for the number of books that have been rated by users
# Users who have rated at least 15 books and the books that have been rated at least 15 users are selected

counts1 = ratings_explicit['userID'].value_counts()
ratings_explicit = ratings_explicit[ratings_explicit['userID'].isin(counts1[counts1 >= 15].index)]
counts = ratings_explicit['bookRating'].value_counts()
ratings_explicit = ratings_explicit[ratings_explicit['bookRating'].isin(counts[counts >= 15].index)]
ratings_explicit.shape


# Data preprocessing up to here!!

(237088, 3)

In [0]:
# Before training the model through matrix factorization, the data (ratings_explicit) is to be spitted into training and testing sets
# This function returns training and test sets along with the raw dataframe for later evaluation of the model 

def informed_train_test(rating_df, train_ratio):
    split_cut = np.int(np.round(rating_df.shape[0] * train_ratio))
    train_df = rating_df.iloc[0:split_cut]
    test_df = rating_df.iloc[split_cut::]
    test_df = test_df[(test_df['userID'].isin(train_df['userID'])) & (test_df['ISBN'].isin(train_df['ISBN']))]
    id_cols = ['userID', 'ISBN']
    trans_cat_train = dict()
    trans_cat_test = dict()
    for k in id_cols:
        cate_enc = preprocessing.LabelEncoder()
        trans_cat_train[k] = cate_enc.fit_transform(train_df[k].values)
        trans_cat_test[k] = cate_enc.transform(test_df[k].values)

    # --- Encode ratings:
    cate_enc = preprocessing.LabelEncoder()
    ratings = dict()
    ratings['train'] = cate_enc.fit_transform(train_df.bookRating)
    ratings['test'] = cate_enc.transform(test_df.bookRating)

    n_users = len(np.unique(trans_cat_train['userID']))
    n_items = len(np.unique(trans_cat_train['ISBN']))


    train = coo_matrix((ratings['train'], (trans_cat_train['userID'], \
                                                          trans_cat_train['ISBN'])) \
                                      , shape=(n_users, n_items))
    test = coo_matrix((ratings['test'], (trans_cat_test['userID'], \
                                                        trans_cat_test['ISBN'])) \
                                     , shape=(n_users, n_items))
    return train, test, train_df

In [0]:
# Fitting the model
train, test, raw_train_df = informed_train_test(ratings_explicit, 0.8)

# Evaluating the model
start_time = time.time()
model=LightFM(no_components=115,learning_rate=0.027,loss='warp')
model.fit(train,epochs=12,num_threads=4)
# with open('saved_model', "wb") as f:
#     pickle.dump(model, f)
auc_train = auc_score(model, train).mean()
auc_test = auc_score(model, test).mean()

print("--- Run time:  {} mins ---".format((time.time() - start_time)/60))
print("Train AUC Score: {}".format(auc_train))
print("Test AUC Score: {}".format(auc_test))

--- Run time:  2.7332381129264833 mins ---
Train AUC Score: 0.9783391356468201
Test AUC Score: 0.9264112710952759


In [0]:
# Generating user-item rating matrix

user_item_matrix = raw_train_df.pivot(index='userID', columns='ISBN', values='bookRating')
user_item_matrix.fillna(0, inplace = True)
user_item_matrix = user_item_matrix.astype(np.int32)
print(user_item_matrix.shape)
user_item_matrix.head()
# with open('saved_matrix', "wb") as f:
#     pickle.dump(user_item_matrix, f)


(3519, 98561)


ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,000104799X,0001048082,0001053736,0001053744,0001055607,0001056107,0001360469,0001374869,000160418X,0001714600,0001821326,0001845039,0001941941,0001941968,0001944711,0001945203,0001952803,0001953877,0001981307,0001981625,0002000547,0002005018,0002005050,0002005557,0002006588,0002111314,0002115328,0002116286,0002118580,0002154900,0002163713,0002165368,0002166828,0002176181,0002176432,...,9997411870,9997412958,9997507002,999750805X,9997512952,9997519086,9997555635,9999980538,B00001U0CP,B0000523SY,B00005B4LM,B00005N5J6,B00005TZWI,B00005UMI4,B00005UMK6,B0000633PU,B000063VMD,B00006CRTE,B00006I4OX,B00006RGIO,B00007FYKW,B00007MF56,B00008RWPV,B000092Q0A,B00009EF82,B00009NDAN,B0000C2W5U,B0000C7BNG,B0000DAPP1,B0000DYXID,B0000T6KHI,B0000T6KIM,B0000VZEJQ,B0000X8HIE,B00011SOXI,B00013AX9E,B0001FZGRQ,B0001GMSV2,B0001I1KOG,B000234N3A
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
243,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
254,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
503,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [0]:
# This function generates user and item dictionaries

def user_item_dikts(interaction_matrix, items_df):
    user_ids = list(interaction_matrix.index)
    user_dikt = {}
    counter = 0 
    for i in user_ids:
        user_dikt[i] = counter
        counter += 1

    item_dikt ={}
    for i in range(items_df.shape[0]):
        item_dikt[(items_df.loc[i,'ISBN'])] = items_df.loc[i,'bookTitle']
    
    return user_dikt, item_dikt

In [0]:
# This function generates top 15 recommendations for the user based on his/her interaction

def similar_recommendation(model, interaction_matrix, user_id, user_dikt, 
                               item_dikt,threshold = 0,number_rec_items = 15):

    #Function to produce user recommendations

    n_users, n_items = interaction_matrix.shape
    user_x = user_dikt[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interaction_matrix.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interaction_matrix.loc[user_id,:][interaction_matrix.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    score_list = scores[0:number_rec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dikt[x]))
    scores = list(pd.Series(score_list).apply(lambda x: item_dikt[x]))
#     jsonScores = json.dumps(scores)
#     print(jsonScores)
    
#     return json.dumps(scores)
    
    print(scores)
    # new dataframe for books
    book_newdf = pd.DataFrame({'bookTitle':scores})
   
    print("Items that were liked by the User:")
    counter = 1
    for i in known_items[:25]:
        print(str(counter) + '- ' + i)
        counter+=1

    print("\n Recommended Items:")
    counter = 1
    for i in scores:
        print(str(counter) + '- ' + i)
        counter+=1
    return book_newdf

In [0]:
user_dikt, item_dikt = user_item_dikts(user_item_matrix, books)

# with open('saved_user_dikt', "wb") as f:
#     pickle.dump(user_dikt, f)
# with open('saved_item_dikt', "wb") as f:
#     pickle.dump(item_dikt, f)
    
    

In [0]:
book_df = similar_recommendation(model, user_item_matrix, 242, user_dikt, item_dikt,threshold = 7)

["Si c'est un homme", "Tout le monde fait l'amour", 'UN Viejo Que Leia Novelas De Amor/the Old Men Who Read Love Stories (ColecciÃ³n Andanzas)', 'Stupid White Men. Eine Abrechnung mit dem Amerika unter George W. Bush', 'Die geliehene Zeit.', 'Russendisko.', 'Le Voyageur imprudent', 'Grand Cahier', 'Le Nom de la rose', 'Oscar et la dame rose', 'Les bruines de Lanester', 'Beatriz et les corps cÃ?Â©lestes', 'Felidae. Roman.', 'Die Verlorene Ehre Der Katharina Blum', 'Per Anhalter Durch Due Galaxis']
Items that were liked by the User:
1- Eine ganz normale AffÃ?Â¤re.
2- Hebt den Dachbalken hoch, Zimmerleute / Seymour wird vorgestellt.
3- Die Korrekturen.
4- QuerschÃ?Â¼sse - Downsize This!
5- Die Liebe in Den Zelten
6- Veronika Deschliesst Zu Sterben / Vernika Decides to Die
7- Familientreffen. ErzÃ?Â¤hlungen.
8- Die Mechanismen der Freude. ErzÃ?Â¤hlungen.
9- Das Kind von morgen. ErzÃ?Â¤hlungen.
10- LÃ?Â¶wenzahnwein. Roman.
11- Das BÃ?Â¶se kommt auf leisen Sohlen.
12- Die Mars- Chroniken. Ro

In [0]:
book_newdf = pd.merge(books, book_df, on='bookTitle')

In [0]:
book_newdf


,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,3442541751,Russendisko.,Wladimir Kaminer,2002.0,Goldmann,http://images.amazon.com/images/P/3442541751.0...,http://images.amazon.com/images/P/3442541751.0...,http://images.amazon.com/images/P/3442541751.0...
1,8472236552,UN Viejo Que Leia Novelas De Amor/the Old Men ...,Luis Sepulveda,1993.0,Tusquets Editores,http://images.amazon.com/images/P/8472236552.0...,http://images.amazon.com/images/P/8472236552.0...,http://images.amazon.com/images/P/8472236552.0...
2,3492045170,Stupid White Men. Eine Abrechnung mit dem Amer...,Michael Moore,2003.0,Piper,http://images.amazon.com/images/P/3492045170.0...,http://images.amazon.com/images/P/3492045170.0...,http://images.amazon.com/images/P/3492045170.0...
3,2253033138,Le Nom de la rose,Umberto Eco,2002.0,LGF,http://images.amazon.com/images/P/2253033138.0...,http://images.amazon.com/images/P/2253033138.0...,http://images.amazon.com/images/P/2253033138.0...
4,2226135022,Oscar et la dame rose,Eric-Emmanuel Schmitt,2002.0,Albin Michel,http://images.amazon.com/images/P/2226135022.0...,http://images.amazon.com/images/P/2226135022.0...,http://images.amazon.com/images/P/2226135022.0...
5,2266022504,Si c'est un homme,Primo Levi,1988.0,Presses Pocket,http://images.amazon.com/images/P/2266022504.0...,http://images.amazon.com/images/P/2266022504.0...,http://images.amazon.com/images/P/2266022504.0...
6,3423011505,Die Verlorene Ehre Der Katharina Blum,Heinrich Boll,1994.0,Deutscher Taschenbuch Verlag,http://images.amazon.com/images/P/3423011505.0...,http://images.amazon.com/images/P/3423011505.0...,http://images.amazon.com/images/P/3423011505.0...
7,3462016407,Die Verlorene Ehre Der Katharina Blum,Heinrich Boll,1994.0,Verlag Kiepenheuer &amp; Witsch GmbH &amp; Co KG,http://images.amazon.com/images/P/3462016407.0...,http://images.amazon.com/images/P/3462016407.0...,http://images.amazon.com/images/P/3462016407.0...
8,3442092981,Felidae. Roman.,Akif Pirincci,1989.0,Goldmann,http://images.amazon.com/images/P/3442092981.0...,http://images.amazon.com/images/P/3442092981.0...,http://images.amazon.com/images/P/3442092981.0...
9,3807701710,Per Anhalter Durch Due Galaxis,Adams,1994.0,"Koch, Neff &amp; Oetinger &amp; Co",http://images.amazon.com/images/P/3807701710.0...,http://images.amazon.com/images/P/3807701710.0...,http://images.amazon.com/images/P/3807701710.0...


In [0]:
export = book_newdf.to_json("/content/drive/My Drive/data/export.json", orient="records")
